In [15]:
%reset

In [16]:
# 加载所需的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
def initialization(n):
    #产生随机仿真的工作表
    joblist = pd.DataFrame(columns=['d_min','d_max','w_o','w_f','p','w','start','stop'],
                           dtype = int)
    #Trapezoidal Suitability Function,列标题代表区块开始的时间点
    tsf = pd.DataFrame(0,index=range(100),columns=range(n))
    for i in range(n):
        #时间段：w,d_min,d_max,d
        d_min = np.random.randint(1,25+1)
        d_max = np.random.randint(d_min,25+1)
        w = np.random.randint(d_max,25+1)
        #时间点：w_o,w_f,start,stop
        w_o = np.random.randint(0,100-w)
        w_f = w_o + w
        #优先级：p
        p = np.random.randint(1,10+1)
        # 更新joblist
        joblist.loc[i] = (d_min,d_max,w_o,w_f,p,w,0,0)

        # 更新打分函数
        s_i = tsf[i].copy()
        # 梯形4个顶点
        a = w_o
        b = int( w_o + np.ceil((w - d_max)/2) )
        c = b + d_max
        d = w_f
        #赋值1
        s_i[range(b,c)] = 1
        #赋值2
        if b > a:
            degree_1 = 1/(2*(b-a))
            n_1 = 1
            for j in range(a,b):
                s_i[j] = degree_1 * (2 * n_1 - 1)
                n_1 += 1
        #赋值3
        if d > c:
            degree_2 = 1/(2*(d-c))
            n_2 = 1
            for j in range(d-1,c-1,-1):
                s_i[j] = degree_2 * (2 * n_2 - 1)
                n_2 += 1

        tsf[i] = s_i

    return tsf,joblist

In [18]:
#初始化
n = 40
np.random.seed(1)
TSF,JOB = initialization(n)

In [19]:
#PD算法：任务按照p值排序
pd_job = JOB.copy()
pd_joblist = JOB.sort_values(by='p',ascending = False).index
pd_joblist

Int64Index([11, 24,  4,  8, 13, 35, 33, 30,  1, 19, 25, 16, 14, 22,  7, 37, 28,
            32, 27, 34,  0, 29, 23, 20, 21, 18, 12,  9,  2,  6, 26,  3, 39, 31,
             5, 36, 17, 15, 10, 38],
           dtype='int64')

In [20]:
def pd_sequencing(n,pd_joblist,pd_job):
    pd_timetable = pd.Series(1,index=range(100))
    empty_timetable = pd.Series(0,index=range(100))
    chosen_job_list = list()

    for i in range(n):
        job_index = pd_joblist[i]
        job_info = pd_job.loc[job_index,]

        d_min = job_info.loc['d_min']
        d_max = JOB.loc[job_index,'d_max']
        w_o = JOB.loc[job_index,'w_o']
        w_f = JOB.loc[job_index,'w_f']
        p = JOB.loc[job_index,'p']
        w = JOB.loc[job_index,'w']

        position_value = pd.Series(0,index=range(100))
        at_least_one = False

        for j in range(w-d_min+1):
            test_start = w_o+j
            test_stop = test_start+d_min
            test_job = empty_timetable.copy()
            test_job.loc[range(test_start,test_stop)]=1
            if np.sum(test_job*pd_timetable) == d_min:
                at_least_one = True
                position_value.loc[test_start] =\
                    np.sum(TSF.loc[range(test_start,test_stop),job_index])

        if at_least_one:
            decide_start = \
                np.min(position_value.index[position_value == position_value.max()])
            decide_stop = decide_start+d_min

            pd_job.loc[job_index,'start'] = decide_start
            pd_job.loc[job_index,'stop'] = decide_stop

            chosen_job_list.append(job_index)

        pd_timetable.loc[range(decide_start,decide_stop)]=0

    return pd_job,chosen_job_list,pd_timetable

In [73]:
pd_job,pd_chosen_joblist,pd_timetable = pd_sequencing(n,pd_joblist,pd_job)
test_pd_job = pd_job.copy()

In [74]:
for i in range(len(pd_chosen_joblist)):
    job_index = pd_chosen_joblist[i]
    job_info = pd_job.loc[job_index,]

    d_min = test_pd_job.loc[job_index,'d_min']
    d_max = test_pd_job.loc[job_index,'d_max']
    w_o = test_pd_job.loc[job_index,'w_o']
    w_f = test_pd_job.loc[job_index,'w_f']
    p = test_pd_job.loc[job_index,'p']
    w = test_pd_job.loc[job_index,'w']
    start = test_pd_job.loc[job_index,'start']
    stop = test_pd_job.loc[job_index,'stop']

    new_start = np.nanmax([pd_job.loc[pd_job.loc[:,'stop']<start,'stop'].max(),
                    w_o,stop-d_max])
    new_stop = np.nanmin([pd_job.loc[pd_job.loc[:,'start']>stop,'start'].min(),
                    w_f,new_start+d_max])

    test_pd_job.loc[job_index,'start'] = new_start
    test_pd_job.loc[job_index,'stop'] = new_stop



In [75]:
empty_timetable1 = pd.Series(0,index=range(100))
empty_timetable2 = pd.Series(0, index=range(100))

for i in range(n):
    empty_timetable1.loc[range(test_pd_job.loc[i,'start'],test_pd_job.loc[i,'stop'])]=i
    empty_timetable2.loc[range(pd_job.loc[i,'start'],pd_job.loc[i,'stop'])]=i
empty_timetable = empty_timetable1-empty_timetable2